In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Importación librerías
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split

In [41]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [11]:
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [ ]:
#Lematizacion 
# Importación librerías
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline

# Importación de librerias
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')

# Definición de la función que tenga como parámetro texto y devuelva una lista de lemas
def split_into_lemmas(text):
    text = text.lower()
    words = text.split()
    return [wordnet_lemmatizer.lemmatize(word) for word in words]

# Creación de matrices de documentos usando CountVectorizer, usando el parámetro 'split_into_lemmas'
vect_lemas = CountVectorizer(stop_words='english', analyzer=split_into_lemmas)

X_dtm = vect_lemas.fit_transform(dataTraining['plot'])
X_dtm.shape

In [ ]:
# Definición de variables predictoras (X)
vect = CountVectorizer(stop_words='english')
X_dtm = vect.fit_transform(dataTraining['plot'])
X_dtm.shape

In [ ]:
#Bigramas 
# Definición de variables predictoras (X)
vect = CountVectorizer(stop_words='english', ngram_range=(1, 3))
X_dtm = vect.fit_transform(dataTraining['plot'])
X_dtm.shape

In [17]:
# Idf 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
tfidf_transformer = TfidfTransformer(use_idf=True, smooth_idf=True, sublinear_tf=False) ##Pruebas 2
vect = CountVectorizer(stop_words='english', ngram_range=(1, 2)) # puse bigramas
X_dtm_1 = vect.fit_transform(dataTraining['plot'])

X_dtm = tfidf_transformer.fit_transform(X_dtm_1)
X_dtm.shape 


(7895, 450140)

In [ ]:
# Definición de variable de interés (y)
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [ ]:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.33, random_state=42)

In [ ]:
%%time
# Definición y entrenamiento
clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=30, random_state=42))
clf.fit(X_train, y_train_genres)

In [ ]:
# Predicción del modelo de clasificación
y_pred_genres = clf.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

# Sin lemas 0.7963860628253788
# COn lemas 0.7983106666028829
# COn bigramas 0.7737576554408515
# Con IDF 0.807300692490283

In [ ]:
# transformación variables predictoras X del conjunto de test
X_test_dtm = vect.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

# Predicción del conjunto de test
y_pred_test_genres = clf.predict_proba(X_test_dtm)

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression
# Definición y entrenamiento
#lr = OneVsRestClassifier(LogisticRegression(random_state=123, solver='lbfgs', max_iter=3000, C=0.01, n_jobs=-1))
# lr = OneVsRestClassifier(LogisticRegression(random_state=123, solver='newton-cg', max_iter=500, C=0.1, n_jobs=-1, penalty='l2')) --- el mejor score 
lr = OneVsRestClassifier(LogisticRegression(random_state=123, solver='saga', max_iter=500, C=1, n_jobs=-1)) 
lr.fit(X_train, y_train_genres)

In [ ]:
# Predicción del modelo de clasificación
y_pred_genres = lr.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

# Sin lematizacion 0.8591001418951785
# Con lematizacion 0.8361467846711504
# Sin lematizar con bigramas 0.8598116682546477
# sin Lematizar con trigramas 0.8590851191421395
# Con iDF 0.8710286526905197 - sag 0.8710200204640789 - newton-cg 0.8710310801528628
# COn IDF sag 500 = 0.8721597443007804
# COn IDF saga 500 C 1 = 0.8763304693871596
# Con IDB y Bigramas saga = 0.8777956356836194
# Ultimo 0.8777956356836194

In [ ]:
%%time

# Definición y entrenamiento
lr2 = OneVsRestClassifier(LogisticRegression(random_state=123, n_jobs=-1))
lr2.fit(X_train, y_train_genres)

# Predicción del modelo de clasificación
y_pred_genres = lr2.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

In [ ]:
%%time
from sklearn.tree import DecisionTreeClassifier

dt = OneVsRestClassifier(DecisionTreeClassifier(random_state=123))
dt.fit(X_train, y_train_genres)
y_pred = dt.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred, average='macro')

In [ ]:
%%time
from sklearn.svm import SVC

svm = OneVsRestClassifier(SVC(random_state=123))
svm.fit(X_train, y_train_genres)
y_pred = svm.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred, average='macro')

In [ ]:
%%time
from xgboost import XGBClassifier

xgb = OneVsRestClassifier(XGBClassifier(random_state=123))
xgb.fit(X_train, y_train_genres)
y_pred = xgb.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred, average='macro')

In [ ]:
%%time
from sklearn.ensemble import AdaBoostClassifier

adaboost = OneVsRestClassifier(AdaBoostClassifier(random_state=123))
adaboost.fit(X_train, y_train_genres)
y_pred = adaboost.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred, average='macro')

In [ ]:
# transformación variables predictoras X del conjunto de test
X_test_dtm_1 = vect.transform(dataTesting['plot'])
X_test_dtm = tfidf_transformer.transform(X_test_dtm_1)

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

# Predicción del conjunto de test
y_pred_test_genres = lr.predict_proba(X_test_dtm)

In [ ]:
# Guardar predicciones en formato exigido en la competencia de kaggle
res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)
res.to_csv('pred_genres_text_LR_TIDF_3.csv', index_label='ID')
res.head()

Ejemplo interesante sobre algo similar..

https://github.com/tomkeith/Multi-label-classification-with-NLP


Ejemplo de análisis 

https://www.kaggle.com/code/saisandeepjallepalli/imdb-5000-eda-and-data-visualization

In [28]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalMaxPooling1D, Conv1D
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam

# Suponiendo que tienes dos listas: 'plots' (las descripciones de las películas)
# y 'genres' (las listas de géneros correspondientes)


In [18]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))

In [19]:
# Procesamiento del texto
maxlen = 100  # longitud de las secuencias
max_words = 10000  # número máximo de palabras a considerar

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(dataTraining['plot'])
sequences = tokenizer.texts_to_sequences(dataTraining['plot'])

word_index = tokenizer.word_index
print(f'{len(word_index)} unique tokens found.')

data = pad_sequences(sequences, maxlen=maxlen)

multilabel_binarizer = MultiLabelBinarizer()
labels = multilabel_binarizer.fit_transform(dataTraining['genres'])

38706 unique tokens found.


In [20]:
data

array([[ 272,   60, 2974, ...,   44, 8503,  148],
       [   0,    0,    0, ..., 1145,  550, 1890],
       [   6, 2813,    3, ...,  114,   70, 1978],
       ...,
       [   0,    0,    0, ...,  138,    4,  487],
       [  17,    1,  653, ...,  157,    1,  341],
       [   0,    0,    0, ...,    4, 4194, 2231]])

In [21]:

# Dividir los datos en un conjunto de entrenamiento y un conjunto de validación
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2)

# Construcción del modelo
model = Sequential()
model.add(Embedding(max_words, 50, input_length=maxlen))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(multilabel_binarizer.classes_), activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

# Entrenamiento del modelo
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))


Epoch 1/10
198/198 [==============================] - 7s 29ms/step - loss: 0.3199 - acc: 0.2137 - val_loss: 0.2860 - val_acc: 0.1906
Epoch 2/10
198/198 [==============================] - 6s 31ms/step - loss: 0.2553 - acc: 0.2867 - val_loss: 0.2645 - val_acc: 0.2717
Epoch 3/10
198/198 [==============================] - 6s 30ms/step - loss: 0.1734 - acc: 0.4580 - val_loss: 0.2598 - val_acc: 0.3008
Epoch 4/10
198/198 [==============================] - 5s 27ms/step - loss: 0.0877 - acc: 0.5380 - val_loss: 0.2945 - val_acc: 0.2932
Epoch 5/10
198/198 [==============================] - 6s 29ms/step - loss: 0.0385 - acc: 0.5434 - val_loss: 0.3502 - val_acc: 0.2989
Epoch 6/10
198/198 [==============================] - 6s 29ms/step - loss: 0.0170 - acc: 0.5500 - val_loss: 0.4023 - val_acc: 0.2894
Epoch 7/10
198/198 [==============================] - 6s 29ms/step - loss: 0.0085 - acc: 0.5416 - val_loss: 0.4514 - val_acc: 0.3015
Epoch 8/10
198/198 [==============================] - 5s 27ms/step - 

In [ ]:
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import nltk

In [37]:
# Tokenizar el texto
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(dataTraining['plot'])
sequences = tokenizer.texts_to_sequences(dataTraining['plot'])
x = pad_sequences(sequences, maxlen=200)

# Transformar las etiquetas en un formato que se pueda alimentar en la red
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(dataTraining['genres'])
y = multilabel_binarizer.transform(dataTraining['genres'])

# Dividir los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# Definir la red
model = Sequential()
model.add(Embedding(5000, 50, input_length=200))
model.add(Conv1D(100, 3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(y.shape[1], activation='sigmoid'))

# Compilar el modelo
model.compile(optimizer=Adam(0.01), loss='binary_crossentropy',  metrics=['acc'])

# Entrenar el modelo
model.fit(x_train, y_train, validation_split=0.1, batch_size=64, epochs=10, validation_data=(x_test, y_test))

# Evaluar el modelo
#y_pred = model.predict(x_test)
#auc = roc_auc_score(y_test, y_pred, average='macro')


Epoch 1/10
87/87 [==============================] - 4s 37ms/step - loss: 0.3080 - acc: 0.2164 - val_loss: 0.2704 - val_acc: 0.2393
Epoch 2/10
87/87 [==============================] - 3s 36ms/step - loss: 0.2457 - acc: 0.2995 - val_loss: 0.2591 - val_acc: 0.2604
Epoch 3/10
87/87 [==============================] - 3s 38ms/step - loss: 0.2095 - acc: 0.3599 - val_loss: 0.2527 - val_acc: 0.2773
Epoch 4/10
87/87 [==============================] - 3s 37ms/step - loss: 0.1725 - acc: 0.4086 - val_loss: 0.2606 - val_acc: 0.2778
Epoch 5/10
87/87 [==============================] - 3s 34ms/step - loss: 0.1407 - acc: 0.4385 - val_loss: 0.2980 - val_acc: 0.2436
Epoch 6/10
87/87 [==============================] - 3s 33ms/step - loss: 0.1112 - acc: 0.4656 - val_loss: 0.3226 - val_acc: 0.2727
Epoch 7/10
87/87 [==============================] - 3s 36ms/step - loss: 0.0853 - acc: 0.4785 - val_loss: 0.3831 - val_acc: 0.2457
Epoch 8/10
87/87 [==============================] - 3s 35ms/step - loss: 0.0665 - a

In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

# Cargamos los datos
#df = pd.read_csv('movies.csv') # Reemplazar con tu archivo

# Convertir las cadenas de lista en listas
dataTraining['genres'] = dataTraining['genres'].apply(eval)

# Inicializar MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Ajustar y transformar los géneros a representación binaria
labels = mlb.fit_transform(dataTraining['genres'])

# Preprocesamiento del texto
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(dataTraining['plot'])
sequences = tokenizer.texts_to_sequences(dataTraining['plot'])
word_index = tokenizer.word_index

# Asegurar que todas las secuencias tengan la misma longitud
data = pad_sequences(sequences, maxlen=500)

# Dividir los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.3)

# Construir el modelo
model = Sequential()
model.add(Embedding(len(word_index) + 1, 128, input_length=500))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(len(labels[0]), activation='sigmoid')) # Cambiar 'softmax' por 'sigmoid'

# Compilar el modelo
model.compile(loss='binary_crossentropy',  # Cambiar 'categorical_crossentropy' por 'binary_crossentropy'
              optimizer='adam',
              metrics=['accuracy'])

# Entrenar el modelo
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_test, y_test))


Epoch 1/10
44/44 [==============================] - 804s 18s/step - loss: 0.3974 - accuracy: 0.1735 - val_loss: 0.2933 - val_accuracy: 0.1933
Epoch 2/10
14/44 [========>.....................] - ETA: 10:37 - loss: 0.2953 - accuracy: 0.2059

KeyboardInterrupt: 